In [1]:
import asyncio
import pandas as pd
from coworker.modules.coworker import Coworker

coworker = Coworker()

In [2]:
question = '''
Can you show me Alphabet's and Meta's assets and liabilities for the past 5 years?
'''

# Create tasks
task_summary = asyncio.create_task(coworker.generate_summary(question))
task_query_and_data = asyncio.create_task(coworker.process_query_and_data(question))

# Await tasks concurrently
summary, (query, data) = await asyncio.gather(task_summary, task_query_and_data)

print(summary)
print('')
print(query)
print('')
if data != 'Apologies. I cannot figure out a way to answer this request.':
    print(pd.DataFrame(data).head())
else:
    print(data)